In [ ]:
#returns 2 tables, document on ad - doc-ad normalized score ; ad strength - ad normalized score
#requires: base table of train (or validation similiar to train) with document id of display id merged (from events)
def ad_on_doc_str(train):
    train_ad_count_per_display = train.groupby(['display_id'])['display_id'].agg({'ads_on_doc' : 'count'}).reset_index()
    train_temp = train.merge(train_ad_count_per_display, how = 'left', on = 'display_id')
    del train_ad_count_per_display
    document_on_ad = train_temp.groupby(['document_id','ad_id','ads_on_doc']).clicked.agg({'clicked' : 'sum', 'total' : 'count'}).reset_index()
    document_on_ad['uni_chance'] = 1 / document_on_ad['ads_on_doc']
    document_on_ad['clicked_percent'] = document_on_ad['clicked'] / document_on_ad['total']
    document_on_ad['clicked_percent_normalized'] = (document_on_ad['clicked'] + 12 * document_on_ad['clicked_percent'].mean()) / (12 + document_on_ad['total'])
    document_on_ad['likelihood_normalized'] = document_on_ad['clicked_percent_normalized'] / document_on_ad['uni_chance']
    document_on_ad['like_mul_total_normalized'] = document_on_ad['likelihood_normalized'] * document_on_ad['total']
    document_on_ad = document_on_ad.drop(document_on_ad.columns[[2,4,5,6,7,8]],axis = 1).groupby(['document_id','ad_id']).sum().reset_index()
    document_on_ad['score'] = document_on_ad['like_mul_total_normalized'] / document_on_ad['total']
    document_on_ad.drop(document_on_ad.columns[[2,3]],axis = 1, inplace=True)
    document_on_ad.sort_values('score',inplace=True, ascending=False)
    ad_strength = train_temp.groupby(['ad_id','ads_on_doc']).clicked.agg({'clicked' : 'sum', 'total' : 'count'}).reset_index()
    ad_strength['uni_chance'] = 1 / ad_strength['ads_on_doc']
    ad_strength['clicked_percent'] = ad_strength['clicked'] / ad_strength['total']
    ad_strength['clicked_percent_normalized'] = (ad_strength['clicked'] + 12 * ad_strength['clicked_percent'].mean()) / (12 + ad_strength['total'])
    ad_strength['likelihood_normalized'] = ad_strength['clicked_percent_normalized'] / ad_strength['uni_chance']
    ad_strength['like_mul_total_normalized'] = ad_strength['likelihood_normalized'] * ad_strength['total']
    ad_strength = ad_strength.drop(ad_strength.columns[[1,3,4,5,6,7]],axis = 1).groupby(['ad_id']).sum().reset_index()
    ad_strength['score'] = ad_strength['like_mul_total_normalized'] / ad_strength['total']
    ad_strength.drop(ad_strength.columns[[1,2]],axis = 1, inplace=True)
    ad_strength.sort_values('score',inplace=True, ascending=False)
    del train_temp
    return document_on_ad, ad_strength